# Pset 3 part 1 of 2: RealSense Point Cloud Registration

The week of 10/22 is the first lab week where you will work with a real robot. More information about working with the robots and lab hours will be announced next week. **Note that only students taking 6.881 for credit may sign up for lab sessions.**

In the last problem set you implemented the basic ICP algorithm and used KMeans clustering for point cloud segmentation to allow the simulated robot to interact with the world. The lab task will be very similar. Your goal is to program the robot to open a cabinet, then pick up an object and put it in the open cabinet.

This problem introduces the physical camera system you'll be working with. In the lab there is a large table. On one end there is a [KUKA iiwa arm](https://www.kuka.com/en-us/products/robotics-systems/industrial-robots/lbr-iiwa), and on three sides (above the robot(?), and on either side of the robot) there are fixed [Intel RealSense D415](https://realsense.intel.com/stereo/) cameras.

With cameras surrounding the table, most of the objects and cabinet will be visible, similar to the scene in pset 2. Unlike last week, however, now these point clouds have noise. As mentioned in class, point clouds from RealSense cameras tend to look "lumpy". Your job is to design a segmentation method to isolate the object of interest, and to implement a more robust version of ICP to successfully match this isolated point cloud to the perfect models.

(An important concept not included in this pset is camera calibration and point cloud alignment. The staff has calibrated the cameras and fused all of the point clouds together for you, but in general this is a non-trivial problem.)

## 0. Set Up the Environment

In [ ]:
# Run this first

%load_ext autoreload

import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

import numpy as np
from realsense_point_cloud_registration import (isolate_table,
                                                plot_meshcat_point_cloud,
                                                segment_drill,
                                                segment_spatula,
                                                make_meshcat_color_array,
                                                align_scene_to_model,
                                                visualize_transform)

In [ ]:
# Start meshcat for visualization - this only has to be run once

# If you interrupt the kernel of this notebook, you'll need to run this cell again to 
# restart the meshcat server, and then refresh the visualization window. 

# This will open a mesh-cat server in the background, click on the url to display 
# visualization in a separate window. 
vis = meshcat.Visualizer()

# Duplicate the visualization inside this cell (at a lower resolution). 
# vis.jupyter_cell()

In [ ]:
# You can use this to clear all meshcat visualizations.
vis.delete()

## 1. Object Segmentation

### 1.1 Spatula Segmentation

In pset 2, you had to segment the bleach bottle and box from the rest of the point cloud for the robot to interact with them. The final segmentations used a variety of methods including area filters and KMeans clustering. One reason why this worked so well is the point cloud was perfect: the table was perfectly flat and the objects were strictly on top of the table. This made it easy to remove everything from below the table and clearly isolate the objects of interest.

Run the cell below to visualize the first point cloud, which has the spatula you looked at in pset 2 on the lab table. What's the difference between this point cloud and the simulated one you used last week?

In [ ]:
scene_points = np.load("point_clouds/scenes/spatula_points.npy")
scene_colors = np.load("point_clouds/scenes/spatula_colors.npy")

# This is a wrapper around creating a meshcat point cloud. See the 
# function in realsense_point_cloud_registration.py for more details.
plot_meshcat_point_cloud(vis, 'scene', scene_points, scene_colors)

The first step we'll do is similar to last week: an area filter. Since the table never moves, we can get rid of everything that's not above the table. We've done this for you. Run the cell below to see the results. Note that it overwrites the full scene cloud in meshcat. You can change the name of the `vis` element to put it in a separate element, and use the meshcat controls (in the top right corner of the meshcat window) to select which elements are visible.

In [ ]:
table_points, table_colors = isolate_table(scene_points, scene_colors)
plot_meshcat_point_cloud(vis, 'scene', table_points, table_colors)


Take a look at the thresholds we're using to filter. We defined the origin of the scene to be just above the surface of the table. Therefore, we expect that removing everything below z = 0 would get rid of the entire table surface and only leave things on top of it. Is that the case? 

Clearly we need to do more to isolate the spatula. Your job is to finish filling out the function `segment_spatula` in realsense\_point\_cloud\_processing.py to isolate the spatula from the rest of the table.

There are many ways of completing this part. Some common segmentation strategies include:

- More detailed XYZ filters
- Color filters in RGB or [HSV](https://en.wikipedia.org/wiki/HSL_and_HSV) space
- Clustering
- Plane detection

In this problem set you will only have to segment a specific point cloud, so your segmentation strategy does not have to generalize for any object or location. 

You may use any one or any combination of the methods listed above, and/or other strategies. There is no one right answer, so as long as your algorithm works you will get full credit.

There are two tests related to this part. The first is a unit test of just your segmentation method. It isn't very strict and has lots of room to be flexible. The second is an integration test with the point cloud alignment optimization you will write in the next part of this pset. We expect your chosen segmentation method to work with your chosen optimization. If you have code that works but doesn't pass both of these tests, let us know on Piazza.

In [ ]:
spatula_points, spatula_colors = segment_spatula(table_points, table_colors)
plot_meshcat_point_cloud(vis, 'scene', spatula_points, spatula_colors)

### 1.2 Drill Segmentation

Now repeat the process for a toy drill. Fill out the function `segment_drill` in realsense\_point\_cloud\_segmentation.py. You can but are not required to use the same methods you did for the spatula. It has a similar testing scheme to the spatula segmentation. Code to visualize the drill scene is repeated below.

In [ ]:
vis.delete()

scene_points = np.load("point_clouds/scenes/drill_points.npy")
scene_colors = np.load("point_clouds/scenes/drill_colors.npy")

# This is a wrapper around creating a meshcat point cloud. See the function
# in realsense_point_cloud_registration.py for more details.
plot_meshcat_point_cloud(vis, 'scene', scene_points, scene_colors)

In [ ]:
table_points, table_colors = isolate_table(scene_points, scene_colors)
plot_meshcat_point_cloud(vis, 'scene', table_points, table_colors)

In [ ]:
drill_points, drill_colors = segment_drill(table_points, table_colors)
plot_meshcat_point_cloud(vis, 'scene', drill_points, drill_colors)

## 2. More Robust Point Cloud Alignment

### 2.1 Running ICP

Now that you have an segmented point clouds, it's time to align them to model objects so the robot can manipulate it. Like in pset 2, we have perfect models, but now our scenes have lots of noise. Try running the basic ICP algorithm you implemented last week on both the drill and the spatula. Play around with running `icp` and `repeat_icp_until_good_fit`. There's a blank copy of iterative_closest_point.py in this pset you can paste your pset 2 code into. We recommend changing the values of the parameters given to the methods.

In [ ]:
# load the models
spatula_model = np.load("point_clouds/models/spatula_model.npy")
drill_model = np.load("point_clouds/models/drill_model.npy")

In [ ]:
# change for the spatula or drill
model = np.copy(spatula_model)
scene = np.copy(spatula_points)

red = make_meshcat_color_array(model.shape[0], 0.5, 0, 0)
plot_meshcat_point_cloud(vis, "spatula_model", model, red)

In [ ]:
from iterative_closest_point import icp, repeat_icp_until_good_fit, visualize_icp

X_MS, error, num_iters = icp(scene, model, max_iterations=20, tolerance=1e-3)

# X_MS, error, num_tries = repeat_icp_until_good_fit(
#     scene, model, error_threshold=1, max_tries=100, max_iterations=20, tolerance=1e-3)

visualize_icp(vis, scene, model, X_MS)

### 2.2 Simplified Point Cloud Alignment

As discussed in class, ICP doesn't work very well with complications such as partial views and noise, both of which are present in our scenes. One way to solve this is by implementing a more robust version of ICP, such as normal ICP. Another possibility is taking advantage of known information about the scene. In this case, we know that the objects will always be lying flat on the table. This means instead of trying to fit a 6 DOF pose of the scene to the model, we only need three dimensions: ($x, $y, $\theta$).

This smaller dimensional alignment has the same goal as ICP to find the best-fit transform between the two point clouds. However we can no longer use the SVD trick to find the rotation matrix, so we will use a different optimization method to solve for $x$, $y$, and $\theta$. The [BFGS algorithm](https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm) takes a similar iterative approach to non-convex optimization. You do not need to be familiar with the algorithm or how it works, just how to use its [scipy implementation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fmin_bfgs.html) for this optimization problem.

Our strategy for solving for the transformation is as follows:

1. Pick an initial guess of the transformation by the centers of the two point clouds and a random orientation

2. Run the BFGS optimization with this initial guess to estimate the true best-fit transform.

3. Repeat steps 1-2 multiple times and take the best estimate of the best-fit transform.

The outer loop to repeat the optimization step is already implemented in `align_scene_to_model` in realsense\_point\_cloud\_registration.py. Most of the inner loop is implemented as well. All you need to do is design the cost function for the BFGS algorithm to minimize, which is the nested `cost` function inside `best_fit_transform`.

Here are the details about the formulation of the optimization problem:

* The transform we want to find contains two parts: a translation in the $xy$-plane and a rotation about the $z$-axis, which can be written as a 4x4 homogenous transformation matrix of the form

$$
    \begin{align}
    \begin{bmatrix}
        \cos(\theta) & -\sin(\theta) & 0 & x \\
        \sin(\theta) & \cos(\theta) & 0 & y \\
        0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 1
    \end{bmatrix}
    \end{align}
$$

* Although there are only three variables we're trying to find, we will use 4 decision variables: $x$, $y$, $\sin(\theta)$, and $\cos(\theta)$. Because the transformation shown above relies on $\sin(\theta)$ and $\cos(\theta)$, it is more straightforward to treat those as separate variables to the solver and use the objective function to show the relationship between them.

* The actual objective we're trying to minimize is the distance between the transformed scene point cloud and the model point cloud. We can use nearest neighbors distances for this comparison. It's important to keep in mind how outliers will affect this distance metric, and if there are any tricks you can use to help curb their effects. (*Hint*: there is a `max_distance` parameter in the method signature you might want to use.)

* There are two point clouds you can use to test your implementation on a non-noisy scene. `spatula_model_transformed.npy` and `drill_model_transformed.npy` both have 10000 points sampled from the spatula and drill models that have undergone a simple and known xy translation and rotation about z. Two of the tests will make sure your algorithm performs well on these point clouds in addition to your segmented ones.

With these points in mind, fill out the `cost` method inside `best_fit_transform` in realsense\_point\_cloud\_registration.py. You can test your code visually by running the cell below. Similar to pset 2, the final transformed scene point cloud is plotted in yellow. In addition to designing the cost function, feel free to experiment with the parameters given to `align_scene_to_model`.

In [ ]:
%reload_ext autoreload
% autoreload 2

X_MS, error = align_scene_to_model(
    scene, model, max_distance=0.05, num_iters=10, num_sample_points=250)

visualize_transform(vis, scene, X_MS)

# Run Tests

Running the cell below will print out a score for every test and the total score.

In [ ]:
import os
# Run the tests
os.popen("python test_pset_3.py test_results.json")

# Print the results json for review
import test_pset_3
print test_pset_3.pretty_format_json_results("test_results.json")